### Set up

#### 1. Set  up  accounts and role

In [13]:
#!pip install sagemaker==1.39.0

In [14]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
account_id =  boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name


#role = sagemaker.get_execution_role()
role="arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20190118T115449".format(account_id)


#### 2. Setup image and instance type

In [15]:
pytorch_custom_image_name="ppi-extractor:gpu-1.0.0-201909140804"
instance_type = "ml.p3.2xlarge" 

In [16]:
docker_repo = "{}.dkr.ecr.{}.amazonaws.com/{}".format(account_id, region, pytorch_custom_image_name)

#### 3. Configure train/ test and validation datasets

In [17]:
bucket = "aegovan-data"

In [18]:
trainfile = "s3://{}/processed_dataset/train_unique_pub_v6_less_negative.json".format(bucket)
testfile= "s3://{}/processed_dataset/test_unique_pub_v6_less_negative.json".format(bucket)
valfile="s3://{}/processed_dataset/val_unique_pub_v6_less_negative.json".format(bucket)
embeddingfile="s3://{}/embeddings/wikipedia-pubmed-and-PMC-w2v.bin.txt".format(bucket)
s3_output_path= "s3://{}/results/".format(bucket)
s3_code_path= "s3://{}/ppi_code/".format(bucket)

### Start training

In [19]:
inputs = {
    "train" : trainfile,
    "val" :valfile,
    "embedding" : embeddingfile
}

In [20]:
hyperparameters = {
    "dataset":"PpiDatasetFactory",
    "trainfile":trainfile.split("/")[-1],
    "valfile":valfile.split("/")[-1],
    "embeddingfile":embeddingfile.split("/")[-1],
    "embeddim":200,
    "batchsize": "32",
    "epochs" : "1000",   
    "log-level" : "INFO",
    "lstmhiddensize": 8,
    "fclayersize": 10,
    "numlayers":3,
    "cnn_output":100,
    "poolingkernelsize":3
   
}

In [21]:
metric_definitions = [{"Name": "TrainLoss",
                     "Regex": "###score: train_loss### (\d*[.]?\d*)"}
                    ,{"Name": "ValidationLoss",
                     "Regex": "###score: val_loss### (\d*[.]?\d*)"}
                    ,{"Name": "TrainFScore",
                     "Regex": "###score: train_fscore### (\d*[.]?\d*)"}
                   ,{"Name": "ValidationFScore",
                     "Regex": "###score: val_fscore### (\d*[.]?\d*)"}
                    ]

In [22]:
git_config = {'repo': 'https://github.com/elangovana/PPI-typed-relation-extractor.git',
              'branch': 'master',
              'commit': 'c050a2818efd6dc6d2adb3dac07c102b94c92eab'}

In [23]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
     entry_point='main_train.py',
                    source_dir = 'source/algorithms',
                    dependencies =['source/algorithms', 'source/datasets',  'source/preprocessor'],
                    role=role,
                    framework_version ="1.0.0",
                    py_version='py3',
                   # git_config= git_config,
                    image_name= docker_repo,
                    train_instance_count=1,
                    train_instance_type=instance_type,
                    hyperparameters =hyperparameters,
                    output_path=s3_output_path,
                    metric_definitions=metric_definitions,
                    #train_use_spot_instances = True
                    code_location=s3_code_path,


                    train_volume_size=30,
                    base_job_name ="ppi-extractor")

In [ ]:
estimator.fit(inputs)

2019-09-28 11:06:01 Starting - Starting the training job...
2019-09-28 11:06:02 Starting - Launching requested ML instances......
2019-09-28 11:07:24 Starting - Preparing the instances for training...
2019-09-28 11:07:58 Downloading - Downloading input data...............
2019-09-28 11:11:11 Training - Downloading the training image..........bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-09-28 11:13:23,354 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-09-28 11:13:23,380 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-09-28 11:13:26,406 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-09-28 11:13:26,675 sagemaker-containers INFO     Module main_train does not provide a setup.py. 
Generating setup.py
2019-09-28 11:13:26,675 sagemaker-containers INFO     Generating setup.cfg
2019-09-28 11:13:26,676 


2019-09-28 11:13:22 Training - Training image download completed. Training in progress.2019-09-28 11:21:12,224 - algorithms.PretrainedEmbedderLoader - INFO - The number of words intialised without embbeder is 126
2019-09-28 11:22:31,395 - algorithms.transform_sentence_tokeniser - INFO - Running sentence tokenisor 
2019-09-28 11:24:35,557 - algorithms.transform_sentence_tokeniser - INFO - Completed  sentence tokenisor 
2019-09-28 11:24:35,565 - algorithms.transform_text_index - INFO - Transforming TransformTextToIndex
2019-09-28 11:24:36,113 - algorithms.transform_text_index - INFO - Completed TransformTextToIndex
2019-09-28 11:24:36,114 - algorithms.transform_sentence_tokeniser - INFO - Running sentence tokenisor 
2019-09-28 11:24:54,329 - algorithms.transform_sentence_tokeniser - INFO - Completed  sentence tokenisor 
2019-09-28 11:24:54,333 - algorithms.transform_text_index - INFO - Transforming TransformTextToIndex
2019-09-28 11:24:54,404 - algorithms.transform_text_index - INFO - C

### HPO

In [ ]:
objective_metric_name ="ValidationFScore"

In [ ]:
from sagemaker.tuner import HyperparameterTuner, IntegerParameter, CategoricalParameter, ContinuousParameter
hyperparameter_ranges = {'lstmhiddensize': IntegerParameter(2,200), #ContinuousParameter(0.01, 0.2),
                        "fclayersize": IntegerParameter(2,50),
                        "numlayers":IntegerParameter(1,10),
                        "poolingkernelsize":IntegerParameter(2,10)
                        }

In [ ]:
hyperparameters={ "trainfile":trainfile.split("/")[-1],
    "valfile":valfile.split("/")[-1],
    "embeddingfile":embeddingfile.split("/")[-1],
                        "embeddim":"200",
                        "dataset":"PpiDatasetFactory",
                 "earlystoppingpatience":20,
                         'epochs': 100}

In [ ]:

estimator = PyTorch(
     entry_point='main_train.py',
                    source_dir = 'source/algorithms',
                    dependencies =['source/algorithms', 'source/datasets', 'source/preprocessor'],
                    role=role,
                    framework_version ="1.0.0",
                    py_version='py3',
                   # git_config= git_config,
                    image_name= docker_repo,
                    train_instance_count=1,
                    train_instance_type=instance_type,
                    hyperparameters =hyperparameters,
                    output_path=s3_output_path,
                    metric_definitions=metric_definitions,
                    code_location=s3_code_path,


                    #train_use_spot_instances = True
                    train_volume_size=30,
                    base_job_name ="hpo-ppi-extractor")

In [ ]:
tuner = HyperparameterTuner(estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=50,
                            max_parallel_jobs=4,
                            strategy="Random",
                            base_tuning_job_name="hpo-ppi-extractor")
tuner.fit(inputs)